# Understanding Python Decorators

This notebook will take you to understand the concept of decorators in python, and how and when to use them. I created this notebook while I was learning the concept of Python Decorators from Python Tips online book [here](https://book.pythontips.com/en/latest/decorators.html).

## What are decorators?
in simple words: they are functions which modify the functionality of other functions. They make the code more concise and more pythonic. 
Some people believe that they are one of the most difficult concepts that anyone can grasp. However, we will try to simplify the concept and help you understand it very well.

## Everything in Python is an object

In [3]:
def hi(name="yasoob"):
    return "hi " + name
print(hi())
print(hi("Mohammed"))

hi yasoob
hi Mohammed


In [5]:
# We can even assign a function to a variable like
greet = hi
# We are not using parentheses here because we are not calling the function hi
# instead we are just putting it into the greet variable. Let's try to run this

print(greet("Ali"))
# output: 'hi yasoob'

hi Ali


In [ ]:
# Let's see what happens if we delete the old hi function!
del hi
print(hi())
#outputs: NameError


In [7]:

print(greet())
#outputs: 'hi yasoob'

hi yasoob


## Defining functions within functions:
So those are the basics when it comes to functions. Let’s take your knowledge one step further. In Python we can define functions inside other functions:

In [8]:
def hi(name="yasoob"):
    print("now you are inside the hi() function")

    def greet():
        return "now you are in the greet() function"

    def welcome():
        return "now you are in the welcome() function"

    print(greet())
    print(welcome())
    print("now you are back in the hi() function")

hi()
#output:now you are inside the hi() function
#       now you are in the greet() function
#       now you are in the welcome() function
#       now you are back in the hi() function

now you are inside the hi() function
now you are in the greet() function
now you are in the welcome() function
now you are back in the hi() function


In [9]:
# This shows that whenever you call hi(), greet() and welcome()
# are also called. However the greet() and welcome() functions
# are not available outside the hi() function e.g:

greet()
#outputs: NameError: name 'greet' is not defined

'hi yasoob'

## Returning functions from within functions:
It is not necessary to execute a function within another function, we can return it as an output as well:

In [10]:
def hi(name="yasoob"):
    def greet():
        return "now you are in the greet() function"

    def welcome():
        return "now you are in the welcome() function"

    if name == "yasoob":
        return greet
    else:
        return welcome

a = hi()
print(a)
#outputs: <function greet at 0x7f2143c01500>

#This clearly shows that `a` now points to the greet() function in hi()
#Now try this

print(a())
#outputs: now you are in the greet() function

<function hi.<locals>.greet at 0x000001A8261ED9D0>
now you are in the greet() function


In [11]:
# We can call the returned function through this way
a = hi()()
print(a)

now you are in the greet() function


## Giving a function as an argument to another function:

In [12]:
def hi():
    return "hi yasoob!"

def doSomethingBeforeHi(func):
    print("I am doing some boring work before executing hi()")
    print(func())

doSomethingBeforeHi(hi)
#outputs:I am doing some boring work before executing hi()
#        hi yasoob!

I am doing some boring work before executing hi()
hi yasoob!


## Writing your first decorator:
In the last example we actually made a decorator! Let’s modify the previous decorator and make a little bit more usable program:

# References
1. Python Tips Book: https://book.pythontips.com/en/latest/decorators.html